# 安徽安庆市项目
## 将MEIC中本地清单的范围设为0
## `Set MEIC to 0 in Anqing`

---
*@author: Evan*\
*@date: 2023-10-05*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from namelist import *
from integrate import *

In [2]:
ncfile = xr.open_dataset(progdir + 'GRIDCRO2D_2023141.nc')
lon = ncfile.LON.squeeze()
lat = ncfile.LAT.squeeze()

shp = gpd.read_file(progdir + 'shapefile/Anqing/Anqing.shp')

mask = polygon_to_mask(shp.geometry[0],lon,lat)
mask_da = xr.DataArray(mask,dims=('ROW','COL'))

In [9]:
sections = ['ind','pow','tra','res','agr']

for sec in sections:
    ds_meic = xr.open_dataset(eval(f'meic_{sec}_file'))
    for var_name,var in ds_meic.items():
        # 将位于mask内的值设为0
        ds_meic[var_name] = ds_meic[var_name].where(~mask_da, 0)
    ds_meic.to_netcdf(datadir + f'step2_meic_and_leaq/emis.CN3AH_135X138.{sec}.ncf')